In [59]:
import os
from PIL import Image
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


# Define directories
train_directory = './Bone_Fracture_Binary_Classification/train/'
test_directory = './Bone_Fracture_Binary_Classification/test/'
valid_directory = './Bone_Fracture_Binary_Classification/val/'

classes = ['Fractured', 'non- Fractured']

# Define transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalizing with ImageNet mean and std
    ]),

    'test': transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    'valid': transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

}

# Create datasets
train_dataset= datasets.ImageFolder(train_directory, transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(test_directory, transform=data_transforms['test'])
valid_dataset= datasets.ImageFolder(valid_directory, transform=data_transforms['valid'])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# device
device = 'cuda' if torch.cuda.is_available else 'cpu'


In [67]:
class XrayClassifierModel(nn.Module):
    def __init__(self) -> None:
        super(XrayClassifierModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,2)
    def forward(self, x):
        x= self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(x)
        x = self.fc3(x)
        return x
        


In [68]:
num_epoch = 20
learning_rate = 0.01
model = XrayClassifierModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer= torch.optim.SGD(model.parameters(), lr=learning_rate)
for x in range(num_epoch):
    for i, (images, labeles) in enumerate(train_loader):
        images, labeles = images.to(device), labeles.to(device)
        pred = model(images)
        loss = criterion(pred, labeles)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if(i % 100 == 0):
            print(f'Epochs: {num_epoch}/{x+1} loss: {loss.item():.4f}')
print('The training Finished')



Epochs: 20/1 loss: 0.7146
Epochs: 20/1 loss: 0.6928
Epochs: 20/1 loss: 0.6883
Epochs: 20/2 loss: 0.6926
Epochs: 20/2 loss: 0.7001
Epochs: 20/2 loss: 0.6960
Epochs: 20/3 loss: 0.6932
Epochs: 20/3 loss: 0.6964
Epochs: 20/3 loss: 0.6953
Epochs: 20/4 loss: 0.6936
Epochs: 20/4 loss: 0.6901
Epochs: 20/4 loss: 0.6886
Epochs: 20/5 loss: 0.6945
Epochs: 20/5 loss: 0.6878
Epochs: 20/5 loss: 0.6912
Epochs: 20/6 loss: 0.6930
Epochs: 20/6 loss: 0.6879
Epochs: 20/6 loss: 0.6891
Epochs: 20/7 loss: 0.6902
Epochs: 20/7 loss: 0.6889
Epochs: 20/7 loss: 0.6893
Epochs: 20/8 loss: 0.6959
Epochs: 20/8 loss: 0.6914
Epochs: 20/8 loss: 0.6905
Epochs: 20/9 loss: 0.6887
Epochs: 20/9 loss: 0.6883
Epochs: 20/9 loss: 0.6820
Epochs: 20/10 loss: 0.6797
Epochs: 20/10 loss: 0.6687
Epochs: 20/10 loss: 0.6644
Epochs: 20/11 loss: 0.6922
Epochs: 20/11 loss: 0.6703
Epochs: 20/11 loss: 0.6735
Epochs: 20/12 loss: 0.6975
Epochs: 20/12 loss: 0.6625
Epochs: 20/12 loss: 0.6233
Epochs: 20/13 loss: 0.5895
Epochs: 20/13 loss: 0.6149
E

In [72]:
correct_no = []
no_test = 0
with torch.no_grad():
    no_correct = 0
    for i, (image2, labele2) in enumerate(test_loader):
        image2, kabeke2 = image2.to(device), labele2.to(device)
        prediction = model(image2)
        print(len(prediction))
        no_test += 32
        for x in range(len(prediction)):
            if(np.argmax(prediction[x].to('cpu'))==labele2[x]):
                correct_no.append(1)


32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
20


In [79]:
print("accuracy is: ", len(correct_no)/ no_test)

accuracy is:  0.75390625
